In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.3.2-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('../Python-and-Spark-for-Big-Data/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', 
                    inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], 
                            outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [16]:
final_data = output.select('features', 'Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 498.22930046349205|
| stddev|  77.21077345737652|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean|  501.6838794287746|
| stddev|  83.93399854627145|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model = lr.fit(train_data)

In [26]:
test_results = lr_model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -6.637338858679925|
|-0.6460738221749693|
|  -17.9155315918739|
|  5.624261337931102|
|-13.825558829750605|
|-22.836872191044336|
| 1.9120046969047735|
| -4.419698333125098|
|  -4.14912347144724|
| -8.852656134934875|
|-2.4198947381570406|
|-2.4618307462237112|
|-18.293957854138455|
|-3.3537206734869756|
|  -6.01023281543246|
|  4.506180515487813|
| 17.505763642192278|
|  5.504207869813513|
|  4.763002829615061|
|-1.3182180126539151|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

10.026113800644419

In [29]:
test_results.r2

0.985639667773023

In [30]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [31]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8162006488763...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.5171218025062...|
|[31.5261978982398...|
|[31.5761319713222...|
|[31.7216523605090...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.9453957483445...|
|[31.9764800614612...|
|[32.0180740106320...|
|[32.0305497162129...|
|[32.0542618511847...|
|[32.0775900443291...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|289.10858457859445|
|[30.5743636841713...| 442.7104875802406|
|[30.8162006488763...| 284.0018725403429|
|[30.9716756438877...| 489.0143484189616|
|[31.0662181616375...|462.75885203742496|
|[31.1239743499119...| 509.7839260308101|
|[31.3091926408918...|430.80871314302885|
|[31.4252268808548...|  535.186416987887|
|[31.5171218025062...|280.06754412183295|
|[31.5261978982398...| 417.9471823272727|
|[31.5761319713222...| 543.6464787274854|
|[31.7216523605090...|350.23875737809635|
|[31.8164283341993...| 519.4164493577948|
|[31.8186165667690...| 449.7723940436226|
|[31.9453957483445...| 663.0301567530844|
|[31.9764800614612...| 326.0882655186124|
|[32.0180740106320...|  340.277347103123|
|[32.0305497162129...| 588.7702755487983|
|[32.0542618511847...| 557.1116548393679|
|[32.0775900443291...|402.35135323175996|
+--------------------+------------